In [1]:
# Libraries for data loading, data manipulation and data visulisation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr


# Libraries for data preparation and model building
# import *

# Setting global constants to ensure notebook results are reproducible
# PARAMETER_CONSTANT = ###

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Load train data
df_train = pd.read_csv("df_train.csv", index_col=0)

In [3]:
#Brief view of data
df_train.head(5)

,time,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,Seville_clouds_all,...,Madrid_temp_max,Barcelona_temp,Bilbao_temp_min,Bilbao_temp,Barcelona_temp_min,Bilbao_temp_max,Seville_temp_min,Madrid_temp,Madrid_temp_min,load_shortfall_3h
0,2015-01-01 03:00:00,0.666667,level_5,0.0,0.666667,74.333333,64.000000,0.0,1.000000,0.0,...,265.938000,281.013000,269.338615,269.338615,281.013000,269.338615,274.254667,265.938000,265.938000,6715.666667
1,2015-01-01 06:00:00,0.333333,level_10,0.0,1.666667,78.333333,64.666667,0.0,1.000000,0.0,...,266.386667,280.561667,270.376000,270.376000,280.561667,270.376000,274.945000,266.386667,266.386667,4171.666667
2,2015-01-01 09:00:00,1.000000,level_9,0.0,1.000000,71.333333,64.333333,0.0,1.000000,0.0,...,272.708667,281.583667,275.027229,275.027229,281.583667,275.027229,278.792000,272.708667,272.708667,4274.666667
3,2015-01-01 12:00:00,1.000000,level_8,0.0,1.000000,65.333333,56.333333,0.0,1.000000,0.0,...,281.895219,283.434104,281.135063,281.135063,283.434104,281.135063,285.394000,281.895219,281.895219,5075.666667
4,2015-01-01 15:00:00,1.000000,level_7,0.0,1.000000,59.000000,57.000000,2.0,0.333333,0.0,...,280.678437,284.213167,282.252063,282.252063,284.213167,282.252063,285.513719,280.678437,280.678437,6620.666667


In [4]:
#function replacee_null_with_mean replace the null values in a dataframe column with its mean value    

def replace_null_with_mean(df, col):
    df_cols = df.columns
    if col not in df_cols:
        raise ValueError
        
    new_df = df.copy()
    mean_val = new_df[col].mean()
    
    def mean_value(val):
        if np.isnan(val) :
            return mean_val
        return val

    
    new_df[col] = new_df[col].apply(mean_value)
#     new_df[col].replace(np.NaN, mean_val)
    
    return new_df

In [5]:
improved_df = replace_null_with_mean(df_train, "Valencia_pressure")

In [6]:
# look at data statistics
improved_df['Valencia_pressure'].isna().sum()

0

In [7]:
# function drop_columns accepts a dataframe, a turple of columns that should be droped, and returns a new dataframe
def drop_columns(df, cols):
    col_name = [col for col in df if col.endswith(cols)]
    accepted_col_name = [col for col in df if col not in col_name]
    df_modified = df[accepted_col_name]
    
    return df_modified

In [8]:
# Remove columns that will generally not affect the response from domain research of renewable energy 
cols_not_needed = ('clouds_all', 'weather_id', 'temp','temp_min')
improved_df = drop_columns(improved_df, cols_not_needed)
improved_df.shape

(8763, 31)

In [9]:
# convert time col to pandas datetime type
def conver_time(df, col):
    return pd.to_datetime(df['time'])
improved_df['time'] = conver_time(improved_df, 'time')
improved_df['time'].dtype

dtype('<M8[ns]')

In [10]:
# Converts columns with object type to numeric data type, and drops the first column
def object_converter(df):
    new_df = df.copy()
    cols = df.columns
    object_type = [col for col in cols if df[col].dtype == 'O']
    new_df = pd.get_dummies(df, columns= object_type,drop_first=True)
    return new_df

In [11]:
improved_df = object_converter(improved_df)
improved_df_2 = improved_df.copy() # Create a copy of the dataset, to enable modeling with two different dataset
improved_df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8763 entries, 0 to 8762
Data columns (total 62 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   time                        8763 non-null   datetime64[ns]
 1   Madrid_wind_speed           8763 non-null   float64       
 2   Bilbao_rain_1h              8763 non-null   float64       
 3   Valencia_wind_speed         8763 non-null   float64       
 4   Seville_humidity            8763 non-null   float64       
 5   Madrid_humidity             8763 non-null   float64       
 6   Bilbao_wind_speed           8763 non-null   float64       
 7   Bilbao_wind_deg             8763 non-null   float64       
 8   Barcelona_wind_speed        8763 non-null   float64       
 9   Barcelona_wind_deg          8763 non-null   float64       
 10  Seville_wind_speed          8763 non-null   float64       
 11  Barcelona_rain_1h           8763 non-null   float64     

In [12]:
improved_df_2.head()

,time,Madrid_wind_speed,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_wind_speed,Bilbao_wind_deg,Barcelona_wind_speed,Barcelona_wind_deg,...,Seville_pressure_sp23,Seville_pressure_sp24,Seville_pressure_sp25,Seville_pressure_sp3,Seville_pressure_sp4,Seville_pressure_sp5,Seville_pressure_sp6,Seville_pressure_sp7,Seville_pressure_sp8,Seville_pressure_sp9
0,2015-01-01 03:00:00,0.666667,0.0,0.666667,74.333333,64.000000,1.000000,223.333333,6.333333,42.666667,...,0,0,1,0,0,0,0,0,0,0
1,2015-01-01 06:00:00,0.333333,0.0,1.666667,78.333333,64.666667,1.000000,221.000000,4.000000,139.000000,...,0,0,1,0,0,0,0,0,0,0
2,2015-01-01 09:00:00,1.000000,0.0,1.000000,71.333333,64.333333,1.000000,214.333333,2.000000,326.000000,...,0,0,1,0,0,0,0,0,0,0
3,2015-01-01 12:00:00,1.000000,0.0,1.000000,65.333333,56.333333,1.000000,199.666667,2.333333,273.000000,...,0,0,1,0,0,0,0,0,0,0
4,2015-01-01 15:00:00,1.000000,0.0,1.000000,59.000000,57.000000,0.333333,185.000000,4.333333,260.000000,...,0,0,1,0,0,0,0,0,0,0


In [13]:
def split_datetime_col(df, col):  
    new_df = df.copy()
    new_df['year'] = new_df[col].dt.year
    new_df['month'] = new_df[col].dt.month
    new_df['week'] = new_df[col].dt.week
    new_df['day'] = new_df[col].dt.day
    new_df['hour'] = new_df[col].dt.hour
    return new_df
improved_df_2 = split_datetime_col(improved_df_2, 'time')
improved_df_2.head()

,time,Madrid_wind_speed,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_wind_speed,Bilbao_wind_deg,Barcelona_wind_speed,Barcelona_wind_deg,...,Seville_pressure_sp5,Seville_pressure_sp6,Seville_pressure_sp7,Seville_pressure_sp8,Seville_pressure_sp9,year,month,week,day,hour
0,2015-01-01 03:00:00,0.666667,0.0,0.666667,74.333333,64.000000,1.000000,223.333333,6.333333,42.666667,...,0,0,0,0,0,2015,1,1,1,3
1,2015-01-01 06:00:00,0.333333,0.0,1.666667,78.333333,64.666667,1.000000,221.000000,4.000000,139.000000,...,0,0,0,0,0,2015,1,1,1,6
2,2015-01-01 09:00:00,1.000000,0.0,1.000000,71.333333,64.333333,1.000000,214.333333,2.000000,326.000000,...,0,0,0,0,0,2015,1,1,1,9
3,2015-01-01 12:00:00,1.000000,0.0,1.000000,65.333333,56.333333,1.000000,199.666667,2.333333,273.000000,...,0,0,0,0,0,2015,1,1,1,12
4,2015-01-01 15:00:00,1.000000,0.0,1.000000,59.000000,57.000000,0.333333,185.000000,4.333333,260.000000,...,0,0,0,0,0,2015,1,1,1,15


### Modeling with Ridge Regression

In [16]:
# Drop time for both data set
improved_df_no_time = improved_df.drop('time', axis=1)
improved_df_2_no_time = improved_df_2.drop('time', axis=1)
improved_df_2_no_time.head()

,Madrid_wind_speed,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_wind_speed,Bilbao_wind_deg,Barcelona_wind_speed,Barcelona_wind_deg,Seville_wind_speed,...,Seville_pressure_sp5,Seville_pressure_sp6,Seville_pressure_sp7,Seville_pressure_sp8,Seville_pressure_sp9,year,month,week,day,hour
0,0.666667,0.0,0.666667,74.333333,64.000000,1.000000,223.333333,6.333333,42.666667,3.333333,...,0,0,0,0,0,2015,1,1,1,3
1,0.333333,0.0,1.666667,78.333333,64.666667,1.000000,221.000000,4.000000,139.000000,3.333333,...,0,0,0,0,0,2015,1,1,1,6
2,1.000000,0.0,1.000000,71.333333,64.333333,1.000000,214.333333,2.000000,326.000000,2.666667,...,0,0,0,0,0,2015,1,1,1,9
3,1.000000,0.0,1.000000,65.333333,56.333333,1.000000,199.666667,2.333333,273.000000,4.000000,...,0,0,0,0,0,2015,1,1,1,12
4,1.000000,0.0,1.000000,59.000000,57.000000,0.333333,185.000000,4.333333,260.000000,3.000000,...,0,0,0,0,0,2015,1,1,1,15


In [17]:
# Split data into predictors and response
X = improved_df_no_time.drop('load_shortfall_3h', axis=1)
y = improved_df_no_time['load_shortfall_3h']
X_2 = improved_df_2_no_time.drop('load_shortfall_3h', axis=1)
y_2 = improved_df_2_no_time['load_shortfall_3h']
X_2.head()

,Madrid_wind_speed,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_wind_speed,Bilbao_wind_deg,Barcelona_wind_speed,Barcelona_wind_deg,Seville_wind_speed,...,Seville_pressure_sp5,Seville_pressure_sp6,Seville_pressure_sp7,Seville_pressure_sp8,Seville_pressure_sp9,year,month,week,day,hour
0,0.666667,0.0,0.666667,74.333333,64.000000,1.000000,223.333333,6.333333,42.666667,3.333333,...,0,0,0,0,0,2015,1,1,1,3
1,0.333333,0.0,1.666667,78.333333,64.666667,1.000000,221.000000,4.000000,139.000000,3.333333,...,0,0,0,0,0,2015,1,1,1,6
2,1.000000,0.0,1.000000,71.333333,64.333333,1.000000,214.333333,2.000000,326.000000,2.666667,...,0,0,0,0,0,2015,1,1,1,9
3,1.000000,0.0,1.000000,65.333333,56.333333,1.000000,199.666667,2.333333,273.000000,4.000000,...,0,0,0,0,0,2015,1,1,1,12
4,1.000000,0.0,1.000000,59.000000,57.000000,0.333333,185.000000,4.333333,260.000000,3.000000,...,0,0,0,0,0,2015,1,1,1,15


In [18]:
# Import scaler method from sklearn
from sklearn.preprocessing import StandardScaler

In [19]:
# Create scaler object
scaler = StandardScaler()

In [24]:
# Create scaled version of the predictors (there is no need to scale the response)
X_scaled = scaler.fit_transform(X)
X_scaled_2 = scaler.fit_transform(X_2)

In [25]:
# Convert the scaled predictor values into a dataframe
X_standardise = pd.DataFrame(X_scaled,columns=X.columns)
X_standardise_2 = pd.DataFrame(X_scaled_2,columns=X_2.columns)
X_standardise_2.head()

,Madrid_wind_speed,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_wind_speed,Bilbao_wind_deg,Barcelona_wind_speed,Barcelona_wind_deg,Seville_wind_speed,...,Seville_pressure_sp5,Seville_pressure_sp6,Seville_pressure_sp7,Seville_pressure_sp8,Seville_pressure_sp9,year,month,week,day,hour
0,-0.950708,-0.362123,-0.796169,0.516117,0.270621,-0.501451,0.630823,1.932284,-1.660205,0.542975,...,-0.210555,-0.221833,-0.142332,-0.223808,-0.146043,-1.226179,-1.602429,-1.709703,-1.675368,-1.090901
1,-1.130863,-0.362123,-0.381412,0.692953,0.298017,-0.501451,0.607959,0.630270,-0.578686,0.542975,...,-0.210555,-0.221833,-0.142332,-0.223808,-0.146043,-1.226179,-1.602429,-1.709703,-1.675368,-0.654451
2,-0.770554,-0.362123,-0.657917,0.383491,0.284319,-0.501451,0.542632,-0.485743,1.520733,0.144442,...,-0.210555,-0.221833,-0.142332,-0.223808,-0.146043,-1.226179,-1.602429,-1.709703,-1.675368,-0.218001
3,-0.770554,-0.362123,-0.657917,0.118238,-0.044439,-0.501451,0.398912,-0.299741,0.925711,0.941509,...,-0.210555,-0.221833,-0.142332,-0.223808,-0.146043,-1.226179,-1.602429,-1.709703,-1.675368,0.218449
4,-0.770554,-0.362123,-0.657917,-0.161751,-0.017043,-0.894581,0.255192,0.816272,0.779762,0.343708,...,-0.210555,-0.221833,-0.142332,-0.223808,-0.146043,-1.226179,-1.602429,-1.709703,-1.675368,0.654899


In [26]:
X_standardise.head()

,Madrid_wind_speed,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_wind_speed,Bilbao_wind_deg,Barcelona_wind_speed,Barcelona_wind_deg,Seville_wind_speed,...,Seville_pressure_sp23,Seville_pressure_sp24,Seville_pressure_sp25,Seville_pressure_sp3,Seville_pressure_sp4,Seville_pressure_sp5,Seville_pressure_sp6,Seville_pressure_sp7,Seville_pressure_sp8,Seville_pressure_sp9
0,-0.950708,-0.362123,-0.796169,0.516117,0.270621,-0.501451,0.630823,1.932284,-1.660205,0.542975,...,-0.200604,-0.220129,4.513665,-0.191182,-0.219843,-0.210555,-0.221833,-0.142332,-0.223808,-0.146043
1,-1.130863,-0.362123,-0.381412,0.692953,0.298017,-0.501451,0.607959,0.630270,-0.578686,0.542975,...,-0.200604,-0.220129,4.513665,-0.191182,-0.219843,-0.210555,-0.221833,-0.142332,-0.223808,-0.146043
2,-0.770554,-0.362123,-0.657917,0.383491,0.284319,-0.501451,0.542632,-0.485743,1.520733,0.144442,...,-0.200604,-0.220129,4.513665,-0.191182,-0.219843,-0.210555,-0.221833,-0.142332,-0.223808,-0.146043
3,-0.770554,-0.362123,-0.657917,0.118238,-0.044439,-0.501451,0.398912,-0.299741,0.925711,0.941509,...,-0.200604,-0.220129,4.513665,-0.191182,-0.219843,-0.210555,-0.221833,-0.142332,-0.223808,-0.146043
4,-0.770554,-0.362123,-0.657917,-0.161751,-0.017043,-0.894581,0.255192,0.816272,0.779762,0.343708,...,-0.200604,-0.220129,4.513665,-0.191182,-0.219843,-0.210555,-0.221833,-0.142332,-0.223808,-0.146043


In [27]:
# Import the ridge regression module from sklearn
from sklearn.linear_model import Ridge

In [29]:
# Create ridge model
ridge = Ridge()
ridge_2 = Ridge()

In [30]:
# Train the model
ridge.fit(X_standardise, y)

Ridge()

In [31]:
ridge_2.fit(X_standardise_2, y_2)

Ridge()

### Test data set

In [32]:
#Load train data
df_test = pd.read_csv("df_test.csv", index_col=0)

In [33]:
clean_test_df = replace_null_with_mean(df_test, "Valencia_pressure")
clean_test_df['Valencia_pressure'].isna().sum()

0

In [34]:
# Remove columns that will generally not affect the response from domain research of renewable energy 
cols_not_needed = ('clouds_all', 'weather_id', 'temp','temp_min')
clean_test_df = drop_columns(clean_test_df, cols_not_needed)
clean_test_df.shape

(2920, 30)

In [35]:
clean_test_df['time'] = conver_time(clean_test_df, 'time')
clean_test_df.head()

,time,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_wind_speed,Bilbao_wind_deg,Barcelona_wind_speed,...,Valencia_snow_3h,Bilbao_pressure,Valencia_pressure,Seville_temp_max,Madrid_pressure,Valencia_temp_max,Valencia_humidity,Barcelona_temp_max,Madrid_temp_max,Bilbao_temp_max
8763,2018-01-01 00:00:00,5.000000,level_8,0.0,5.000000,87.000000,71.333333,3.000000,193.333333,4.000000,...,0,1025.666667,1013.148351,284.483333,1030.000000,287.483333,46.333333,287.816667,280.816667,285.150000
8764,2018-01-01 03:00:00,4.666667,level_8,0.0,5.333333,89.000000,78.000000,3.666667,143.333333,4.666667,...,0,1026.666667,1013.148351,282.483333,1030.333333,284.150000,53.666667,284.816667,280.483333,284.150000
8765,2018-01-01 06:00:00,2.333333,level_7,0.0,5.000000,89.000000,89.666667,2.333333,130.000000,4.000000,...,0,1025.333333,1013.148351,280.816667,1030.666667,282.816667,42.000000,284.483333,276.483333,282.150000
8766,2018-01-01 09:00:00,2.666667,level_7,0.0,5.333333,93.333333,82.666667,5.666667,196.666667,2.333333,...,0,1025.000000,1013.148351,281.150000,1032.333333,283.483333,45.000000,284.150000,277.150000,284.483333
8767,2018-01-01 12:00:00,4.000000,level_7,0.0,8.666667,65.333333,64.000000,10.666667,233.333333,4.666667,...,0,1022.333333,1013.148351,287.150000,1032.333333,287.150000,41.000000,287.483333,281.150000,286.816667


In [36]:
clean_test_df_copy = object_converter(clean_test_df)
clean_test_df_copy_2 = clean_test_df_copy.copy() # Create a copy of the dataset, to enable modeling with two different dataset
clean_test_df_copy_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2920 entries, 8763 to 11682
Data columns (total 61 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   time                        2920 non-null   datetime64[ns]
 1   Madrid_wind_speed           2920 non-null   float64       
 2   Bilbao_rain_1h              2920 non-null   float64       
 3   Valencia_wind_speed         2920 non-null   float64       
 4   Seville_humidity            2920 non-null   float64       
 5   Madrid_humidity             2920 non-null   float64       
 6   Bilbao_wind_speed           2920 non-null   float64       
 7   Bilbao_wind_deg             2920 non-null   float64       
 8   Barcelona_wind_speed        2920 non-null   float64       
 9   Barcelona_wind_deg          2920 non-null   float64       
 10  Seville_wind_speed          2920 non-null   float64       
 11  Barcelona_rain_1h           2920 non-null   float64 

In [37]:
clean_test_df_copy_2_time = split_datetime_col(clean_test_df_copy_2, 'time')
clean_test_df_copy_2_time.head()

,time,Madrid_wind_speed,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_wind_speed,Bilbao_wind_deg,Barcelona_wind_speed,Barcelona_wind_deg,...,Seville_pressure_sp5,Seville_pressure_sp6,Seville_pressure_sp7,Seville_pressure_sp8,Seville_pressure_sp9,year,month,week,day,hour
8763,2018-01-01 00:00:00,5.000000,0.0,5.000000,87.000000,71.333333,3.000000,193.333333,4.000000,176.666667,...,0,0,0,0,0,2018,1,1,1,0
8764,2018-01-01 03:00:00,4.666667,0.0,5.333333,89.000000,78.000000,3.666667,143.333333,4.666667,266.666667,...,0,0,0,0,0,2018,1,1,1,3
8765,2018-01-01 06:00:00,2.333333,0.0,5.000000,89.000000,89.666667,2.333333,130.000000,4.000000,263.333333,...,0,0,0,0,0,2018,1,1,1,6
8766,2018-01-01 09:00:00,2.666667,0.0,5.333333,93.333333,82.666667,5.666667,196.666667,2.333333,86.666667,...,0,0,0,0,0,2018,1,1,1,9
8767,2018-01-01 12:00:00,4.000000,0.0,8.666667,65.333333,64.000000,10.666667,233.333333,4.666667,283.333333,...,0,0,0,0,0,2018,1,1,1,12


In [38]:
predict_test_data_time = clean_test_df['time']
predict_test_data_time

8763    2018-01-01 00:00:00
8764    2018-01-01 03:00:00
8765    2018-01-01 06:00:00
8766    2018-01-01 09:00:00
8767    2018-01-01 12:00:00
                ...        
11678   2018-12-31 09:00:00
11679   2018-12-31 12:00:00
11680   2018-12-31 15:00:00
11681   2018-12-31 18:00:00
11682   2018-12-31 21:00:00
Name: time, Length: 2920, dtype: datetime64[ns]

In [39]:
## Drop the time colunm for both data set
clean_test_df_copy = clean_test_df_copy.drop('time', axis =1)
clean_test_df_copy_2_time = clean_test_df_copy_2_time.drop('time', axis =1)

In [41]:
predict_result = ridge.predict(clean_test_df_copy)
predict_result

array([124119.77890401, 106200.60842433, 112186.45445591, ...,
       183954.46418979, 180489.13872312, 116271.39097973])

In [42]:
my_dict = {
    'time': predict_test_data_time,
    'load_shortfall_3h':predict_result
}

In [43]:
new_pandas = pd.DataFrame(my_dict)
new_pandas.shape

(2920, 2)

In [44]:
new_pandas.to_csv('file_4.csv',index=False)